In [36]:
import numpy as np
import matplotlib.pyplot as plt
import MolecularDynamics
import Clustering
from importlib import reload
reload(MolecularDynamics)
reload(Clustering)
from MolecularDynamics import *
from Clustering import *

pos_flat = np.loadtxt('lj10clusters.txt')
positions = pos_flat.reshape(-1,pos_flat.shape[1]//2,2)


[[False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]]
[[False False]
 [False False]
 [False False]
 [ True  True]
 [ True  True]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]]
[[False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]
 [False False]]
